# RCFM Model

## 1. Import modules and helper functions

In [2]:
# Modules
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from scipy.optimize import curve_fit

# Helper functions from DataAid.py and DataImport.py
import DataAid
import DataImporter

# Numerically stable class of functions from Neros_v2.py
import Neros_v3

NameError: name 'MW_phi' is not defined

## 2. Load Galaxy Data

In [ ]:
# Load Galaxy Data
sparcGalaxies = DataAid.GetGalaxyData("data/Sparc/Rotmod_LTG/")
littleDataGalaxies = DataAid.GetGalaxyData("data/little-data-things/data/")
lcmGalaxies = DataAid.GetGalaxyData("data/LCMFits/data/")

# Load Milky Way Model Data
xueSofueGalaxies = DataAid.GetGalaxyData("data/XueSofue/")
mcGaughMW = DataAid.GetGalaxyData("data/McGaugh/")

# Create array of Milky Way radius and vlum tuples from model data
MWXueSofue = np.array(xueSofueGalaxies['MW_lum'])
MWMcGaugh = np.array(mcGaughMW['MW_lumMcGaugh'])


## 3. Create Neros class instance

In [ ]:
# Create Neros instance to perform calculations with the supplied Milky Way model as comparison
# Change Milky Way model by changing the variable in the parentheses
# i.e. neros_fns = Neros_v2.Neros(MWModelVariable)

neros_fns = Neros_v3.Neros(MWXueSofue)
MW_name = "MWXueSofue" # Change this if you change the MW model in neros_fns!
MW_rad = neros_fns.mwRad
MW_vLum = neros_fns.mwVLum

MW_vLum_interp_func = neros_fns.mw_vLum_interp

## 4. Designate outfile name, initialize variables, and designate galaxy sample

In [ ]:
# This is where the fitted alpha & vlum free parameter will be written for each galaxy
out_file = "imported-data/data_" + str(MW_name) + ".csv"
with open(out_file, 'w') as f:
    f.write('{0},{1},{2},{3}\n'.format("Galaxy", "Chisquare",  "Alpha", "Beta"))
f.close()

vLcmList = []

avgChiSquared = 0.0
avgPhiZero = 0.0

# This designates which galaxy sample to fit
galaxies = sparcGalaxies

## 5. Fit galaxies, print and save graphs

In [ ]:
"""
This is the main body of the model. 
It loops through galaxies in the designated sample,
it calculates vLCM, vNeros, and chiSquared,
it fits galaxy data using alpha and vLumFreeParam,
it prints the values of alpha and vLumFreeParam to the console,
and it saves the graphs to a file.
"""
for galaxyName in galaxies:
    f, ax = plt.subplots(1, figsize = (15, 15))
    galaxy = np.array(galaxies[galaxyName])
    galaxy_rad = galaxy[:,0]
    galaxy_vObs = galaxy[:,1]
    galaxy_error = galaxy[:,2]
    galaxy_gas = galaxy[:,3]
    galaxy_disk = galaxy[:,4]
    galaxy_bulge = galaxy[:,5]

    # calculate vlumsquared
    galaxy_vLumSquared = neros_fns.vLumSquared(galaxy_gas, galaxy_disk, galaxy_bulge)
    galaxy_vLum = np.sqrt(galaxy_vLumSquared)
    
    # Make sure the arrays are matching lengths
    maxMWRad = MW_rad[len(MW_rad)-1]
    for i in range(len(galaxy_rad)):
        if galaxy_rad[i] > maxMWRad:
            galaxy_rad = galaxy_rad[:i]
            galaxy_vLum = galaxy_vLum[:i]
            galaxy_vObs = galaxy_vObs[:i]
            galaxy_gas = galaxy_gas[:i]
            galaxy_disk = galaxy_disk[:i]
            galaxy_bulge = galaxy_bulge[:i]
            galaxy_error = galaxy_error[:i]
            break
            
    # Find fittedAlpha and fittedVLumFreeParam from curve_fit function
    vals, cov = curve_fit(neros_fns.curve_fit_fn,(galaxy_rad, galaxy_gas, galaxy_disk, galaxy_bulge),
                          galaxy_vObs,sigma=galaxy_error)
    fittedAlpha, fittedVLumFreeParam, disk_scale, bulge_scale = vals

    # Calculate vLCM and vNeros
    vLcm = neros_fns.vLCM(galaxy_rad, galaxy_vLum)
    vLcmList.append(vLcm)
    vNeros = neros_fns.curve_fit_fn((galaxy_rad, galaxy_gas, galaxy_disk, galaxy_bulge),
                                    fittedAlpha, fittedVLumFreeParam, disk_scale, bulge_scale)

    # Calculate chi squared
    chiSquare = neros_fns.chiSquared(galaxy_vObs, vNeros, galaxy_error)
    avgChiSquared = avgChiSquared + chiSquare

    # printing to file & console
    print(f"galaxyName is: {galaxyName}")

    print(f"chiSquare is: {chiSquare}")

    print(f"fittedAlpha is: {fittedAlpha}, \nfittedVLumFreeParam is: {fittedVLumFreeParam}")
    
    print(f"disk_scale is: {disk_scale}, \nbulge_scale is: {bulge_scale}")
    
    print(f"avgChiSquared / len(lcmGalaxies) is: {avgChiSquared / len(lcmGalaxies)}")

    print("--------------------------")

    # Write galaxyName, fittedAlpha, fittedVLumFreeParam to file
    with open(out_file, 'a') as f:
        f.write('{0},{1},{2},{3}\n'.format(galaxyName, chiSquare, fittedAlpha, fittedVLumFreeParam))
    f.close()

    # Calculate galaxy_vObs plus error and put into list
    galaxy_vObs_err_incl = []
    for i in range(len(galaxy_vObs)):
        galaxy_vObs_err_incl.append(galaxy_vObs[i] + galaxy_error[i])

    # calculate new vLum
    galaxy_vLum_updated = []
    for i in range(len(galaxy_vLum)):
        galaxy_vLum_updated.append(galaxy_vLum[i] * fittedVLumFreeParam)

    # plot
    # y-axis scales to the maximum velocity value in the list galaxy_vObs_err_incl
    # or to the maximum value in the list vNeros, whichever is the bigger number
    y_max = max(max(galaxy_vObs_err_incl), max(vNeros))
    ax.set_ylim(bottom = 0, top = y_max + 15)

    # plot vObs and vNeros and updated vLum
    ax.plot(galaxy_rad, vNeros, label="{}_vNeros".format(galaxyName))
    ax.plot(galaxy_rad, galaxy_vObs, label="{}_vObs".format(galaxyName))
    ax.plot(galaxy_rad, galaxy_vLum_updated, label="{}_new_vLum".format(galaxyName))

    # error bar in vObs
    for i in range(len(galaxy_rad)):
        ax.vlines(galaxy_rad[i], galaxy_vObs[i] - galaxy_error[i], galaxy_vObs[i] + galaxy_error[i])

    ax.legend(loc="upper right")

    graph_file_name = "graphs/" + str(galaxyName) + "_" + str(MW_name)

    plt.savefig(graph_file_name)